# Preliminaries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score

In [ ]:
dataset = pd.read_csv("healthcare-dataset-stroke-data.csv")

print(dataset.shape)
dataset.head()

(5110, 12)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


# Preprocessing

In [ ]:
dataset = dataset.drop(columns=['id'])
dataset = dataset[dataset['bmi'].notna()]
dataset = dataset[dataset.smoking_status != "Unknown"]
# dataset = dataset[dataset.gender != "Other"]

In [ ]:
x = dataset.drop(columns=['stroke'])
y = dataset['stroke']

## Random oversampling

In [ ]:
y.value_counts()

0    3246
1     180
Name: stroke, dtype: int64

## Shaping dataset

In [ ]:
oneHotEncoding = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0,4,5,6,9])], remainder='passthrough')
x = np.array(oneHotEncoding.fit_transform(x))
print(x)

[[  0.     1.     0.   ...   1.   228.69  36.6 ]
 [  0.     1.     0.   ...   1.   105.92  32.5 ]
 [  1.     0.     0.   ...   0.   171.23  34.4 ]
 ...
 [  1.     0.     0.   ...   0.   125.2   40.  ]
 [  1.     0.     0.   ...   0.    82.99  30.6 ]
 [  0.     1.     0.   ...   0.   166.29  25.6 ]]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [ ]:
y_train_1 = to_categorical(y_train)
y_test_1 = to_categorical(y_test)
y_1 = to_categorical(y)

# MLP training

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, early_stopping=True, validation_fraction=0.15, verbose=True)

In [ ]:
mlp.fit(x_train, y_train_1)

Iteration 1, loss = 11.95895784
Validation score: 0.501217
Iteration 2, loss = 1.38981544
Validation score: 0.934307
Iteration 3, loss = 1.26575667
Validation score: 0.939173
Iteration 4, loss = 1.24886529
Validation score: 0.939173
Iteration 5, loss = 1.10954388
Validation score: 0.939173
Iteration 6, loss = 0.93212807
Validation score: 0.929440
Iteration 7, loss = 0.79278711
Validation score: 0.902676
Iteration 8, loss = 0.72037018
Validation score: 0.905109
Iteration 9, loss = 0.63675681
Validation score: 0.927007
Iteration 10, loss = 0.56710901
Validation score: 0.914842
Iteration 11, loss = 0.50415954
Validation score: 0.924574
Iteration 12, loss = 0.44542486
Validation score: 0.927007
Iteration 13, loss = 0.39819371
Validation score: 0.931873
Iteration 14, loss = 0.37937242
Validation score: 0.934307
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(early_stopping=True, max_iter=1000, validation_fraction=0.15,
              verbose=True)

In [ ]:
y_pred = mlp.predict(x_test)

In [ ]:
accuracy_score(np.argmax(y_test_1, axis=1), np.argmax(y_pred, axis=1))

0.9358600583090378

In [ ]:
print(classification_report(np.argmax(y_test_1, axis=1), np.argmax(y_pred, axis=1), zero_division=1))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       642
           1       1.00      0.00      0.00        44

    accuracy                           0.94       686
   macro avg       0.97      0.50      0.48       686
weighted avg       0.94      0.94      0.90       686



In [ ]:
y_pred = mlp.predict(x)

In [ ]:
print(classification_report(np.argmax(y_1, axis=1), np.argmax(y_pred, axis=1), zero_division=1))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3246
           1       1.00      0.00      0.00       180

    accuracy                           0.95      3426
   macro avg       0.97      0.50      0.49      3426
weighted avg       0.95      0.95      0.92      3426

